`El resto de este capítulo trata sobre como personalizar casi todos los componentes en tf.keras(por ejemplo Layers,Models and Training Loops). Y para obtener detalles ir al libro o al cuaderno respectivo al Chapter12`

## **Tensors and operations**

Un `tensor` es una estructura de datos multidimensional, normalmente un
array multidimensional (exactamente como un ndarray de NumPy) y son la unidad básica de datos en TensorFlow que se utilizan para construir modelos de aprendizaje automático. Los tensores en TensorFlow son **inmutables**, lo que significa que una vez que se crea un tensor, no se puede cambiar. En cambio, las operaciones en los tensores crean nuevos tensores.

Puede tener cualquier número de dimensiones, desde 0 (un escalar) hasta n (un tensor n-dimensional). Por ejemplo, un tensor 2D se puede utilizar para representar una imagen en escala de grises, donde cada elemento del tensor representa un píxel en la imagen.

In [1]:
import tensorflow as tf
t= tf.constant([[1., 2., 3.], [4., 5., 6.]])# matrix
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [2]:
dim =t.shape # devuelve TensorShape([2, 3])
dim[0]

2

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

>Cuando el **tensor** solo contien un escalar, shape devulve una tupla vacia

### Indexing

In [4]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [72]:
t[..., 1, tf.newaxis]  # equivalente a t[..., 1, tf.newaxis]


<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

### Ops

The `@ operator` was added in Python 3.5, for matrix multiplication: it is
equivalent to calling the tf.matmul() function.

In [ ]:
t + 10
tf.square(t)
t @ tf.transpose(t)

La operación `tf.reduce_sum()` se llama
así porque su kernel GPU (es decir, la implementación GPU) utiliza un
algoritmo de reducción que no garantiza el orden en que se suman los
elementos: como los flotantes de 32 bits tienen una precisión limitada, el
resultado puede cambiar ligeramente cada vez que se llama a esta
operación. Lo mismo ocurre con `tf.reduce_mean()` (pero, por
supuesto, `tf.reduce_max()` es determinista).

### Using `keras.backend`

La API de Keras tiene su propia API de bajo nivel, ubicada en
keras.backend. Incluye funciones como *square()*, *exp()*, y *sqrt()*.
En tf.keras, estas funciones generalmente sólo llaman a las
operaciones correspondientes de TensorFlow. Si quieres
escribir código que sea portable a otras implementaciones de
Keras, deberías usar estas funciones de Keras. Sin embargo, sólo
cubren un subconjunto de todas las funciones disponibles en
TensorFlow.
> por lo que en este libro utilizaremos directamente las operaciones de TensorFlow. 

In [7]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### From/To NumPy

Los tensores se llevan bien con NumPy: puedes crear un tensor a partir
de un array NumPy, y viceversa. Incluso puedes aplicar operaciones
TensorFlow a matrices NumPy y operaciones NumPy a tensores

**NOTA:** NumPy utiliza 64 bits de precisión por defecto, mientras que TensorFlow
utiliza 32-bit. Esto se debe a que la precisión de 32 bits es generalmente más que suficiente
para las redes neuronales, además de que se ejecuta más rápido

In [8]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [9]:
# obtenr el arreglo de un tensor 
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [10]:
# convertir un tensor a arreglo
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [56]:
# operacion de tensor a un nd-array
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [73]:
# operacion de numpy a un tensor 
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Conflicting Types
TensorFlow no realiza ninguna conversión de tipos automáticamente: simplemente lanza una excepción si
intentas ejecutar una operación en tensores con tipos incompatibles. 

In [14]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [59]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Strings

In [60]:
# Convertir un string a tensor de string
tf.constant(b"hello world")

<tf.Tensor: shape=(), dtype=string, numpy=b'hello world'>

In [70]:
# crear un tensor de entero aociado a un string
u = tf.constant([ord(c) for c in "café"])
u

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233])>

In [68]:
# convertir un tensor de enteros a uno de strings 
tf.strings.unicode_encode(u, "UTF-8")

<tf.Tensor: shape=(), dtype=string, numpy=b'caf\xc3\xa9'>

In [66]:
# crear un tensor con la longitud de una cadena 
tf.strings.length("café",unit="UTF8_CHAR")

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [74]:
# String arrays
p = tf.constant(["Café", "Coffee", "caffè", "咖啡"])
p

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'Caf\xc3\xa9', b'Coffee', b'caff\xc3\xa8',
       b'\xe5\x92\x96\xe5\x95\xa1'], dtype=object)>

In [76]:
# crear un tensor con los length de cada string en el tensor 
tf.strings.length(p, unit="UTF8_CHAR")

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 6, 5, 2])>

### Sparse tensors

In [83]:
s = tf.SparseTensor(
    indices=[[0, 1], [1, 0], [2, 3]],  # s.indices es un tensor con los indices
    values=[1., 2., 3.],               # s.values es un tensor con los valores en cada indices
    dense_shape=[3, 4])                # s.dense_shape es un tensor con el shape
print(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [92]:
tf.sparse.to_dense(s) 

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

In [88]:
s2 = s * 2.0

In [96]:
s3 = s + 1.

TypeError: unsupported operand type(s) for +: 'SparseTensor' and 'float'

In [99]:
s4 = tf.constant([[10., 20.], [30., 40.], [50., 60.], [70., 80.]])
tf.sparse.sparse_dense_matmul(s, s4)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 30.,  40.],
       [ 20.,  40.],
       [210., 240.]], dtype=float32)>

In [108]:
s5 = tf.SparseTensor(indices=[[0, 2], [0, 1]],
                     values=[1., 2.],
                     dense_shape=[3, 4])
print(s5)

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]], shape=(2, 2), dtype=int64), values=tf.Tensor([1. 2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


como indices=[[0, 2], [0, 1]], tiene los indices ordenados no sepuede convertir a una matriz densa hay q reordenar los indices

In [111]:
s6 = tf.sparse.reorder(s5)
tf.sparse.to_dense(s6)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 2., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Variables

Un **tf.Variable** es un tipo especial de tensor que se utiliza para almacenar y actualizar valores en memoria durante el entrenamiento de un modelo de ML. A diferencia de los tensores regulares, los **tf.Variable** se pueden modificar y actualizar durante la ejecución del programa

In [114]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [115]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [117]:
v[0, 1:].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42., 42.],
       [ 8., 10., 12.]], dtype=float32)>

In [118]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [122]:
# no se puede usar el simbolo de =, en su lugar es el método assign()

v[1].assign([7., 8., 9.])
try:
    v[1] = ([7., 8., 9.])

except TypeError as ex:
    print(ex)


'ResourceVariable' object does not support item assignment


In [123]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  7.,   8., 200.]], dtype=float32)>

In [124]:
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

### Tensor Arrays
Son listas de tensores. Tienen un tamaño fijo por defecto, pero
opcionalmente pueden hacerse dinámicas. Todos los tensores que
contienen deben tener la misma forma y tipo de datos

In [130]:
array = tf.TensorArray(dtype=tf.float32, size=3)
array = array.write(0, tf.constant([1., 2.]))
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))

In [132]:
# notar que se lee un tensor y se saca de la lista(es decir se queda con valores cero)
array.read(0)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [136]:
# convertir el TensorArray en un solo tensor
array.stack()

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.,  0.],
       [ 3., 10.],
       [ 5.,  7.]], dtype=float32)>

Calcular la varianza y  la media 

In [134]:
mean, variance = tf.nn.moments(array.stack(), axes=0)
mean

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.6666667, 5.6666665], dtype=float32)>

In [135]:
variance

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 4.2222223, 17.555555 ], dtype=float32)>